# Init

In [1]:
import context
from config import robotConfig
from UR_RTDE_wrapper import RTDEwrapper
print( "Loaded!" )

Loaded: /home/scott/dev_rmstudio/rmlib/rmlib/rmtools Exists?: True
Loaded: /home/scott/dev_rmstudio/rmlib Exists?: True
Loaded!


In [3]:
robotConfig['my_components']['arm_config']['ip_address'] = "192.168.0.101"
rtde = RTDEwrapper( robotConfig['my_components']['arm_config'], None )
print( "Created!" )

Created!


# Test Status

In [3]:
var = rtde.get_status_bits()
print( "get_status_bits:", var, type( var ) )

get_status_bits: 0b11 <class 'str'>


In [6]:
var = rtde.get_safety_status_bits()
print( "get_safety_status_bits:", var, type( var ) )

get_safety_status_bits: 0b1 <class 'str'>


In [5]:
var = rtde.get_tcp_pose_vec()
print( "get_tcp_pose_vec:", var, type( var ) )

get_tcp_pose_vec: [-0.1459477794168426, -0.3153451021102023, 0.19125039797344479, -0.008701077917144308, 3.1138871938502275, 0.019909065786114632] <class 'list'>


In [4]:
var = rtde.get_tcp_pose()
print( "get_tcp_pose:\n", var, '\n', type( var ) )

get_tcp_pose:
 [[-0.99960231 -0.00575209  0.02760697 -0.1459508 ]
 [-0.00539814  0.9999025   0.0128782  -0.31533694]
 [-0.02767836  0.01272405 -0.9995359   0.1912343 ]
 [ 0.          0.          0.          1.        ]] 
 <class 'numpy.ndarray'>


In [10]:
var = rtde.get_tcp_force()
print( "get_tcp_force:\n", var, '\n', type( var ) )

get_tcp_force:
 [-4.29530225713023, -8.8445346868358, -5.594714070026336, 2.7039305767777537, -0.49619902071205385, 0.5813904255737924] 
 <class 'list'>


In [8]:
var = rtde.get_joint_angles()
print( "get_joint_angles:\n", var, '\n', type( var ) )

var = rtde.get_joint_degrees()
print( "get_joint_degrees:\n", var, '\n', type( var ) )

get_joint_angles:
 [0.8009634613990784, -1.9169562498675745, 1.7536840438842773, -1.3782809416400355, -1.5533941427813929, -0.774698559437887] 
 <class 'list'>
get_joint_degrees:
 [45.89182588235682, -109.83350262863769, 100.47869431400414, -78.96968093929095, -89.00292830171621, -44.386957850655676] 
 <class 'list'>


In [9]:
var = rtde.enable_freedrive()
print( "enable_freedrive:\n", var, '\n', type( var ) )

enable_freedrive:
 True 
 <class 'bool'>


In [10]:
var = rtde.disable_freedrive()
print( "disable_freedrive:\n", var, '\n', type( var ) )

disable_freedrive:
 True 
 <class 'bool'>


# Test Motion

In [11]:
import numpy as np

orig = rtde.get_tcp_pose()
print( "get_tcp_pose:\n", orig )

trgt = orig.copy()
trgt[2,3] += 0.050

print( "New target pose:\n", trgt )

print( "Diff:\n", np.subtract( trgt, orig ) )

rtde.move_speed( trgt, move_type = 'l', speed = 0.125, accel = 0.25,
                 radius = 0, stop_condition = 'dummy', blocking = True )

get_tcp_pose:
 [[-0.99960434 -0.00576407  0.02753087 -0.14596495]
 [-0.00541185  0.99990278  0.01285105 -0.31533313]
 [-0.02760226  0.01269697 -0.99953834  0.19126315]
 [ 0.          0.          0.          1.        ]]
New target pose:
 [[-0.99960434 -0.00576407  0.02753087 -0.14596495]
 [-0.00541185  0.99990278  0.01285105 -0.31533313]
 [-0.02760226  0.01269697 -0.99953834  0.24126315]
 [ 0.          0.          0.          1.        ]]
Diff:
 [[0.   0.   0.   0.  ]
 [0.   0.   0.   0.  ]
 [0.   0.   0.   0.05]
 [0.   0.   0.   0.  ]]


True

# Test Condition (Blocking)

## First test without condition

In [4]:
import time
orig = rtde.get_tcp_pose()
trgt = orig.copy()
trgt[2,3] += 0.050

bgn = time.time()
rtde.move_speed( trgt, move_type = 'l', speed = 0.012, accel = 0.125,
                 radius = 0, stop_condition = 'dummy', blocking = True )
print( "Blocking move required", time.time()-bgn , "seconds to execute" )

Blocking move required 4.462336778640747 seconds to execute


## Test with Timeout condition (Blocking)

In [5]:
# Test a condition that belongs to an object, which is often the case
import time
from time import sleep

orig = rtde.get_tcp_pose()
trgt = orig.copy()
trgt[2,3] += 0.100


class OutsideCondition:
    """ Simplest timeout condition """
    
    def __init__( self, TO_s ):
        self.TO  = TO_s
        self.bgn = time.time()
        
    def cond( self ):
        if time.time() - self.bgn >= self.TO:
            print( "TIMEOUT!" )
            return True
        else:
            return False
        

TO = OutsideCondition( 2.0 )

bgn = time.time()
rtde.move_speed( trgt, move_type = 'l', speed = 0.012, accel = 0.125,
                 radius = 0, stop_condition = TO.cond, blocking = True )
print( "Blocking move required", time.time()-bgn , "seconds to execute" )

TIMEOUT!
Blocking move required 2.0767970085144043 seconds to execute


# Test Condition (Non-Blocking)

In [6]:
# Test a condition that belongs to an object, which is often the case
import time
from time import sleep

orig = rtde.get_tcp_pose()
trgt = orig.copy()
trgt[2,3] += 0.100


class OutsideCondition:
    """ Simplest timeout condition """
    
    def __init__( self, TO_s ):
        self.TO  = TO_s
        self.bgn = time.time()
        
    def cond( self ):
        if time.time() - self.bgn >= self.TO:
            print( "TIMEOUT!" )
            return True
        else:
            return False
        
TO = None
TO = OutsideCondition( 2.0 )

bgn = time.time()
rtde.move_speed( trgt, move_type = 'l', speed = 0.012, accel = 0.125,
                 radius = 0, stop_condition = TO.cond, blocking = False )
print( "Blocking move required", time.time()-bgn , "seconds to execute" )

Blocking move required 0.13744091987609863 seconds to execute
TIMEOUT!
COND MET
